# RCP-SSP Scenario Analysis (Clean)

This notebook provides a clean implementation to visualize RCP-SSP impacts on climate factors using polars + plotly.

- Data source: `data_parquet/rcp_ssp/all_climate_data.parquet`
- Variables: Pe (mm), ET0 (mm), ETc (mm), IWR (mm)
- Scenarios: RCP2.6-SSP1, RCP4.5-SSP2, RCP8.5-SSP5 (corrected)
- Crop groups: `Sheet1` for Pe/ET0, `Five_crops` for ETc/IWR



In [ ]:
import polars as pl
import plotly.graph_objects as go
from pathlib import Path

# Paths
PARQUET_DIR = Path("../data_parquet/rcp_ssp")

# Load and merge climate data with temperature data
df = pl.read_parquet(PARQUET_DIR / "all_climate_data.parquet")
tas = pl.read_parquet(PARQUET_DIR / "tas_data.parquet")
df = pl.concat([df, tas])  # Use pl.concat to merge DataFrames

# Scenario mapping and filtering
sc_map = {
    "ssp126_corrected": "RCP2.6-SSP1",
    "ssp245_corrected": "RCP4.5-SSP2",
    "ssp585_corrected": "RCP8.5-SSP5",
}
df = df.with_columns(pl.col("Scenario").replace(sc_map).alias("Scenario"))
df = df.filter(pl.col("Scenario").is_in(list(sc_map.values())))

# Colors and variable->crop mapping
SCENARIO_COLORS = {
    "RCP2.6-SSP1": "#2ECC71",
    "RCP4.5-SSP2": "#3498DB",
    "RCP8.5-SSP5": "#E74C3C",
}
VAR_CROP = {"Pe": "Sheet1", "ET0": "Sheet1", "TAS": "Sheet1"}
VARIABLES = ["Pe", "ET0", "TAS"]

## Viz ET0 and Pe

In [ ]:
def plot_ts_ci(df, variable):
    crop = VAR_CROP.get(variable)
    data = df.filter(
        (pl.col("Variable") == variable) & (pl.col("CropType") == crop)
    ).drop_nulls("Value")
    if data.height == 0:
        fig = go.Figure()
        fig.update_layout(title=f"No data: {variable}", template="plotly_white")
        return fig
    yearly = (
        data.group_by(["Year", "Scenario"])
        .agg([pl.col("Value").mean().alias("mean"), pl.col("Value").std().alias("std")])
        .sort(["Scenario", "Year"])
    )
    fig = go.Figure()
    for scn in yearly["Scenario"].unique().to_list():
        sd = yearly.filter(pl.col("Scenario") == scn)
        years = sd["Year"].to_list()
        means = sd["mean"].to_list()
        stds = [0.0 if s is None else s for s in sd["std"].to_list()]
        color = SCENARIO_COLORS.get(scn, "#1ABC9C")
        upper = [m + s for m, s in zip(means, stds)]
        lower = [m - s for m, s in zip(means, stds)]
        fig.add_trace(
            go.Scatter(
                x=years + years[::-1],
                y=upper + lower[::-1],
                fill="toself",
                fillcolor=color,
                opacity=0.2,
                line=dict(color="rgba(255,255,255,0)"),
                showlegend=False,
            )
        )
        fig.add_trace(
            go.Scatter(
                x=years,
                y=means,
                mode="lines",
                name=scn,
                line=dict(color=color, width=3),
            )
        )
    fig.update_layout(
        title=f"{variable} Time Series by Scenario",
        xaxis_title="Year",
        yaxis_title=f"{variable} (mm)",
        template="plotly_white",
        width=1200,
        height=500,
    )
    return fig


for v in VARIABLES:
    plot_ts_ci(df, v).show()

页面的

## 获取现在和未来的对比

In [ ]:
from typing import Optional


def calculate_climate_change(
    df: pl.DataFrame,
    scenario: str,
    variable: str,
    crop_type: Optional[str] = None,
    baseline_year: int = 2020,
) -> dict:
    """
    Calculate mean values before and after a baseline year and percentage change.

    Parameters
    ----------
    df : pl.DataFrame
        Climate data with columns: Year, Scenario, Value, Variable, CropType
    scenario : str
        Climate scenario name (e.g., 'RCP2.6-SSP1')
    variable : str
        Climate variable (e.g., 'Pe', 'ET0', 'ETc', 'IWR', 'TAS')
    crop_type : str, optional
        Crop type. If None, uses default mapping
    baseline_year : int, default=2020
        Split year for before/after comparison

    Returns
    -------
    dict with keys: mean_before, mean_after, change_percent
    """
    # Default crop type mapping
    default_mapping = {
        "Pe": "Sheet1",
        "ET0": "Sheet1",
        "TAS": "Sheet1",
        "ETc": "Five_crops",
        "IWR": "Five_crops",
    }
    crop_type = crop_type or default_mapping.get(variable, "Sheet1")

    # Filter and calculate
    filtered = df.filter(
        (pl.col("Scenario") == scenario)
        & (pl.col("Variable") == variable)
        & (pl.col("CropType") == crop_type)
    ).drop_nulls("Value")

    before = filtered.filter(pl.col("Year") < baseline_year)["Value"].mean()
    after = filtered.filter(pl.col("Year") >= baseline_year)["Value"].mean()
    change = ((after - before) / before * 100) if before else None

    return {
        "scenario": scenario,
        "variable": variable,
        "mean_before": before,
        "mean_after": after,
        "change_percent": change,
    }


def calculate_all_variables(
    df: pl.DataFrame,
    scenario: str,
    variables: list[str] = None,
    baseline_year: int = 2020,
) -> pl.DataFrame:
    """
    Calculate climate change for all variables under a given scenario.

    Parameters
    ----------
    df : pl.DataFrame
        Climate data
    scenario : str
        Climate scenario (required, e.g., 'RCP2.6-SSP1')
    variables : list[str], optional
        Variables to analyze. If None, uses all available variables
    baseline_year : int, default=2020
        Split year for before/after comparison

    Returns
    -------
    pl.DataFrame with columns: Scenario, Variable, MeanBefore, MeanAfter, ChangePercent
    """
    variables = variables or df["Variable"].unique().to_list()
    results = []

    for variable in variables:
        try:
            results.append(
                calculate_climate_change(
                    df, scenario, variable, baseline_year=baseline_year
                )
            )
        except Exception:
            continue

    return pl.DataFrame(results) if results else pl.DataFrame()

In [ ]:
# 单个情景的单个变量
result = calculate_climate_change(df, "RCP2.6-SSP1", "Pe")
result

# 单个情景的所有变量
results = calculate_all_variables(df, "RCP2.6-SSP1")
results

In [ ]:
# 对比多个情景
for scenario in ["RCP2.6-SSP1", "RCP4.5-SSP2", "RCP8.5-SSP5"]:
    print(f"\n{'='*60}")
    print(f"情景: {scenario}")
    print(f"{'='*60}")
    scenario_results = calculate_all_variables(
        df, scenario, variables=["Pe", "ET0", "TAS"]
    )
    display(scenario_results)

## Water Availability

In [ ]:
from scripts.query_scenarios import ScenarioQuery

query = ScenarioQuery("../data_parquet")

In [ ]:
query.list_variables()

In [ ]:
query.param_cols

In [ ]:
from scripts.viz_helpers import quick_plot

filters = {
    "Climate change scenario switch for water yield": 2,
}

quick_plot(
    query,
    variable="yrb_available_surface_water",
    filters=filters,
    time_range=(2020, 2100),
)

In [ ]:
# 重新导入简化版本
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd()))

# 查询数据
data = query.get_series(
    variables="yrb_available_surface_water",
    filters=filters,
    time_range=(2020, 2100),
)


def calculate_transfer_volume(year):
    """根据年份计算南水北调水量"""
    if year < 2014:
        return 0
    elif year < 2050:
        return 85.37  # 一期工程
    else:
        return 174 + 85.37  # 乐观情景：一期+西线


# 使用 map_elements 应用函数
data = data.with_columns(
    [
        pl.col("time")
        .map_elements(calculate_transfer_volume, return_dtype=pl.Float64)
        .alias("SNWTP water transfer volume")
    ]
)

# 计算南水北调后的水量
data = data.with_columns(
    [
        (pl.col("value") + pl.col("SNWTP water transfer volume") / 10).alias(
            "value_snwtp"
        )
    ]
)
data.head()

In [ ]:
def plot_water_comparison_polars(data, title="YRB Available Surface Water + SNWTP"):
    """
    绘制基线水量和南水北调后水量的对比 - 纯 Polars 版本
    """
    # 基线水量统计
    baseline_stats = (
        data.group_by("time")
        .agg(
            [
                pl.col("value").mean().alias("baseline_mean"),
                pl.col("value").std().alias("baseline_std"),
            ]
        )
        .with_columns(
            [
                (pl.col("baseline_mean") - 1.96 * pl.col("baseline_std")).alias(
                    "baseline_ci_lower"
                ),
                (pl.col("baseline_mean") + 1.96 * pl.col("baseline_std")).alias(
                    "baseline_ci_upper"
                ),
            ]
        )
        .sort("time")
    )

    # 南水北调后水量统计
    snwtp_stats = (
        data.group_by("time")
        .agg(
            [
                pl.col("value_snwtp").mean().alias("snwtp_mean"),
            ]
        )
        .sort("time")
    )

    # 创建图表
    fig = go.Figure()

    # 基线水量 - 置信区间
    fig.add_trace(
        go.Scatter(
            x=baseline_stats["time"].to_list(),
            y=baseline_stats["baseline_ci_upper"].to_list(),
            fill=None,
            mode="lines",
            line=dict(color="#2E86AB", width=0),
            showlegend=False,
            hoverinfo="skip",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=baseline_stats["time"].to_list(),
            y=baseline_stats["baseline_ci_lower"].to_list(),
            fill="tonexty",
            mode="lines",
            line=dict(color="#2E86AB", width=0),
            fillcolor="rgba(46, 134, 171, 0.2)",
            name="Baseline 95% CI",
        )
    )

    # 基线水量 - 实线
    fig.add_trace(
        go.Scatter(
            x=baseline_stats["time"].to_list(),
            y=baseline_stats["baseline_mean"].to_list(),
            mode="lines",
            line=dict(color="#2E86AB", width=3),
            name="Baseline (Natural Water)",
        )
    )

    # 南水北调后水量 - 虚线
    fig.add_trace(
        go.Scatter(
            x=snwtp_stats["time"].to_list(),
            y=snwtp_stats["snwtp_mean"].to_list(),
            mode="lines",
            line=dict(color="#FF6B6B", width=2.5, dash="dash"),
            name="With SNWTP",
        )
    )

    # 更新布局
    fig.update_layout(
        title=dict(text=title, font=dict(size=18)),
        xaxis_title="Year",
        yaxis_title="Water Availability (亿 m³)",
        hovermode="x unified",
        height=700,
        width=1200,
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(255,255,255,0.9)",
        ),
    )

    return fig


# 使用
fig = plot_water_comparison_polars(data)
fig.show()

## 大河对比

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_excel("../data/rivers.xlsx", sheet_name="Sheet1")
# Create Figure 1: Discharge bubble chart
x = data["basin"]  # Drainage basin area
y = data["River Length (km)"]  # River length
s = data["New_GRDC"]  # Discharge
rivers = data["River"]
per = data["WRperCaptia"]  # Water resources per capita

# Calculate discharge in 10^8 m^3/a
discharge_annual = s * 60 * 60 * 24 * 365 / 10**8

# Create custom hover text
hover_text = [
    f"<b>{river}</b><br>"
    + f"Basin area: {basin:.0f} km²<br>"
    + f"River length: {length:.0f} km<br>"
    + f"Discharge: {discharge:.1f}×10⁸ m³/a<br>"
    + f"Per capita: {pc:.0f} m³/a"
    for river, basin, length, discharge, pc in zip(rivers, x, y, discharge_annual, per)
]

# Create the bubble chart
fig1 = go.Figure()

# Add main scatter trace
fig1.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode="markers",
        marker=dict(
            size=s**0.5 / 3,  # Adjust size scaling for Plotly
            color=np.log10(per),  # Use log scale for color
            colorscale=[
                [0, "#E8C872"],
                [0.4, "#FFF3CF"],
                [0.6, "#C9D7DD"],
                [1, "#637A9F"],
            ],
            showscale=True,
            colorbar=dict(
                title=dict(text="Annual discharge<br>per capita (m³/a)", side="right"),
                tickmode="array",
                tickvals=[3, 4, 5],
                ticktext=["10³", "10⁴", "10⁵"],
                x=1.02,
            ),
            line=dict(width=0),
        ),
        text=hover_text,
        hovertemplate="%{text}<extra></extra>",
        name="Rivers",
    )
)

# (Interactive) remove static reference markers for discharge legend

# Highlight Yellow River
yellow_idx = rivers[rivers == "Yellow River"].index[0]
yellow_x = x[yellow_idx]
yellow_y = y[yellow_idx]

# Add vertical line for Yellow River
fig1.add_shape(
    type="line",
    x0=yellow_x,
    y0=0,
    x1=yellow_x,
    y1=yellow_y,
    line=dict(color="#8E2D30", width=1, dash="dash"),
)

# Add horizontal line for Yellow River
fig1.add_shape(
    type="line",
    x0=0,
    y0=yellow_y,
    x1=yellow_x,
    y1=yellow_y,
    line=dict(color="#8E2D30", width=1, dash="dash"),
)

# Add Yellow River annotation
fig1.add_annotation(
    x=750,
    y=6100,
    text="Yellow River",
    showarrow=False,
    font=dict(size=12, color="#8E2D30"),
)

# Add discharge value annotation for Yellow River
fig1.add_annotation(
    x=280,
    y=5750,
    text=f"Discharge =<br>{discharge_annual[yellow_idx]:.1f}×10⁸ m³/a",
    showarrow=False,
    font=dict(size=10, color="#8E2D30"),
    align="left",
)

# Update layout
fig1.update_layout(
    title=dict(text="River Discharge Analysis", x=0.5, xanchor="center"),
    xaxis=dict(
        title="Drainage basin area (km²)",
        range=[10, 6200],
        showgrid=True,
        gridcolor="rgba(187, 187, 187, 0.6)",
        gridwidth=0.75,
        griddash="dot",
    ),
    yaxis=dict(
        title="River length (km)",
        range=[1010, 6990],
        showgrid=True,
        gridcolor="rgba(187, 187, 187, 0.6)",
        gridwidth=0.75,
        griddash="dot",
    ),
    plot_bgcolor="white",
    width=900,
    height=600,
    font=dict(family="Arial, sans-serif", size=14),
    hovermode="closest",
)

fig1.show()

In [ ]:
def calculate_future_change(
    data: pl.DataFrame,
    baseline_year: float = 2020,
    value_col: str = None,
    time_col: str = None,
) -> dict:
    """
    Calculate change between baseline and future periods with flexible column names.

    Parameters
    ----------
    data : pl.DataFrame
        Time series data
    baseline_year : float, default=2020
        Year to split before/after periods
    value_col : str, optional
        Value column name. Auto-detects 'Value' or 'value'
    time_col : str, optional
        Time column name. Auto-detects 'Year', 'time', or 'step'

    Returns
    -------
    dict with keys: mean_before, mean_after, change_percent, change_absolute
    """
    # Auto-detect column names
    cols = data.columns
    if value_col is None:
        value_col = "value" if "value" in cols else "Value"
    if time_col is None:
        time_col = "time" if "time" in cols else ("Year" if "Year" in cols else "step")

    before = data.filter(pl.col(time_col) <= baseline_year)[value_col].mean()
    after = data.filter(pl.col(time_col) > baseline_year)[value_col].mean()

    change_abs = after - before if (before is not None and after is not None) else None
    change_pct = ((after - before) / before * 100) if before and before != 0 else None

    return {
        "mean_before": before,
        "mean_after": after,
        "change_absolute": change_abs,
        "change_percent": change_pct,
        "baseline_year": baseline_year,
    }


# 计算现在/未来的变化（函数会自动检测列名）
change = calculate_future_change(data, baseline_year=2020)
change